In [1]:
# libraries
import pandas as pd
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math, time

# local functions
from utils import import_matrix

## Loading data

In [2]:
root    = './'
city    = 'SiouxFalls'

In [3]:
# Importing the networks into a Pandas dataframe consists of a single line of code
# but we can also make sure all headers are lower case and without trailing spaces

netfile = os.path.join(root, city, city + '_net.tntp')
network = pd.read_csv(netfile, skiprows=8, sep='\t')

trimmed= [s.strip().lower() for s in network.columns]
network.columns = trimmed

network.drop(['~', ';'], axis=1, inplace=True)

# rename init_node to from and term_node to to
# check if column has init_node and term_node
if 'init_node' in network.columns:
    network.rename(columns={'init_node': 'from', 'term_node': 'to'}, inplace=True)
    

In [4]:
linkindexordering = list(zip(network['from'], network['to']))
edge_attr = {}
mlt = False
for row in network.itertuples():
    edge_attr[(row[1], row[2])] = {'capacity':row[3], 'length':row[4], 'FFT':row[5], 'alpha':row[6],
                                    'beta':row[7], 'type':row[10], 'flow':0, 'cost':row[5], 'Oflow':None}
    
edge = [(row[1], row[2]) for row in network.itertuples()]

In [5]:
NetGraph = nx.DiGraph()
NetGraph.add_edges_from(edge)
nx.set_edge_attributes(NetGraph, edge_attr)

In [6]:
ODfile = os.path.join(root, city, city + '_trips.tntp')
matrix, index = import_matrix(ODfile)
# convert matrix dictionary to dataframe
OD_raw = pd.DataFrame.from_dict(matrix, orient='index')
ODdemand = OD_raw.stack().reset_index()
ODdemand.columns = ['Origin', 'Destination', 'Demand']

ODdemand = {(row['Origin'], row['Destination']):row['Demand'] for index, row in ODdemand.iterrows()}

In [7]:
zones2cent = {zone: [zone] for zone in OD_raw.columns.to_list()}
#zone2cent = {zone: [row['osmid'] for index, row in centroids[centroids['zoneID'] == zone].iterrows()] for zone in centroids['zoneID']}

## All or nothing assignment

In [8]:
def AONloading(G, zone2cent, Demand, compute_sptt = True):
    '''
    Input:
    G: a directed graph object, which represents a road network.
    zone2cent: a dictionary with zones as keys and cetroids in zone as a list of ints
    zones: a list of zones
    Demand: a dictonary of demand values for each origin-destination pair in the graph.
    Process:
    It computes the SPTT by running the Dijkstra's shortest path algorithm on the graph with the centroids as the origin nodes and the edges' cost attribute as the weight. For each origin-destination pair in the centlist, it finds the shortest path and multiplies it by the demand between the origin and destination.
    If the computexbar flag is True, the function also computes the edge flow (x_bar) for all the edges in the graph by adding the demand between the origin and destination to the edge flow for each edge in the shortest path.
    Output:
    It returns the shortest path travel time (SPTT) and the edge flow (x_bar) for all the edges in the graph.
    '''
    x_bar = {l: 0 for l in G.edges()}
    spedges = {}
    EODTT = {}
    SPTT = 0
    for ozone, onodes in zone2cent.items():
        sspso = [nx.single_source_dijkstra(G, onode, weight='cost') for onode in onodes]
        for dzone, dnodes in zone2cent.items():
            if ozone == dzone:
                continue
            try:
                dem = Demand[(ozone,dzone)]
            except KeyError:
                continue
            if dem <= 0:
                continue
            tempssps = [(sspso[i][0][dnode], sspso[i], dnode) for i in range(len(sspso)) for dnode in dnodes
                        if dnode in sspso[i][0].keys()]
            tempssps = sorted(tempssps, key=lambda x:x[0])
            tt, ssps, dest = tempssps[0]
            path = ssps[1][dest]
            if compute_sptt:
                SPTT += tt*dem
                spedges[(ozone, dzone)] = path
                EODTT[(ozone, dzone)] = tt
            for i in range(len(path)-1):
                x_bar[(path[i], path[i+1])] += dem
    return SPTT, x_bar, spedges, EODTT

In [9]:
G_AON = NetGraph.copy()
SPTT, x_bar, spedges, EODTT = AONloading(G_AON, zones2cent, ODdemand, compute_sptt=True)
print('SPTT: ', SPTT)
print('x_bar: ', x_bar)
print('spedges: ', spedges)
print('EODTT: ', EODTT)

SPTT:  3176000.0
x_bar:  {(1, 2): 3800.0, (1, 3): 6000.0, (2, 1): 3800.0, (2, 6): 6600.0, (3, 1): 6000.0, (3, 4): 10200.0, (3, 12): 6800.0, (6, 2): 6600.0, (6, 5): 10900.0, (6, 8): 16900.0, (4, 3): 10200.0, (4, 5): 13700.0, (4, 11): 7400.0, (12, 3): 6800.0, (12, 11): 11300.0, (12, 13): 12000.0, (5, 4): 14400.0, (5, 6): 10200.0, (5, 9): 7000.0, (11, 4): 6800.0, (11, 10): 19000.0, (11, 12): 11300.0, (11, 14): 17400.0, (9, 5): 7000.0, (9, 8): 800.0, (9, 10): 17000.0, (8, 6): 17600.0, (8, 7): 8100.0, (8, 9): 800.0, (8, 16): 14700.0, (7, 8): 8000.0, (7, 18): 13200.0, (18, 7): 13100.0, (18, 16): 14800.0, (18, 20): 11600.0, (16, 8): 15500.0, (16, 10): 28100.0, (16, 17): 26700.0, (16, 18): 14100.0, (10, 9): 17100.0, (10, 11): 21300.0, (10, 15): 11200.0, (10, 16): 28200.0, (10, 17): 0, (15, 10): 13600.0, (15, 14): 10800.0, (15, 19): 19000.0, (15, 22): 20200.0, (17, 10): 0, (17, 16): 26700.0, (17, 19): 21900.0, (14, 11): 14600.0, (14, 15): 8700.0, (14, 23): 10400.0, (13, 12): 12100.0, (13, 24): 

## Task:
Develop the MSA assignment code that evaulates user equilibrium assignment given the SiouxFalls network and demand



## Work flow to develop MSA
Initial run
* Run dijkstra
* Run AON loading assignment (AON function)
* Update network costs/traveltimes
* Run Dijkstra
* Check relative gap between TSTT and SPTT - Gap is (AON volumes * new costs) / SPTT. SPTT is all demand * min(new costs)

Iterative
* Run AON again
* Calculate MSA volumes/flows
* Update network costs/traveltimes 
* Run dijkstra again
* Check relative gap between TSTT and SPTT - Gap is (MSA volumes * new costs) / SPTT. SPTT is all demand * min(new costs)

In [10]:
def MSA(graph, zone2centroid, demand, max_iterations=100, convergence_threshold=0.05):
    """
    Perform User Equilibrium (UE) Assignment using the Method of Successive Averages (MSA).

    Parameters:
        graph (nx.DiGraph): Directed graph representing the network, with 'cost' as edge weights.
        zone2centroid (dict): Mapping of zones to lists of centroid nodes within each zone.
        demand (dict): OD demand as a dictionary {(origin_zone, destination_zone): demand}.
        max_iterations (int): Maximum number of iterations to run.
        convergence_threshold (float): Threshold for the relative gap to check convergence.

    Returns:
        dict: Final edge flows (volumes) for each edge as {(u, v): flow}.
        list: Convergence history of relative gaps.
    """
    # Initialize edge flows and cost attributes
    for u, v, data in graph.edges(data=True):
        data['flow'] = 0  # Initialize edge flows to zero
        if 'cost' not in data:
            raise ValueError("Graph edges must have an initial 'cost' attribute.")
    
    relative_gaps = []  # To track convergence

    # Step 1: Initial All-or-Nothing (AON) assignment
    print("Running initial AON assignment...")
    SPTT, x_bar, _, _ = AONloading(graph, zone2centroid, demand, compute_sptt=True)

    for iteration in range(1, max_iterations + 1):
        print(f"Iteration {iteration}...")

        # Update flows using MSA: x = x + (1 / iteration) * (x_bar - x)
        for (u, v) in graph.edges():
            graph[u][v]['flow'] = (
                graph[u][v]['flow'] + (1 / iteration) * (x_bar.get((u, v), 0) - graph[u][v]['flow'])
            )

        # Update travel times/costs on the network based on new flows
        update_edge_costs(graph)

        # Step 2: Run shortest path (Dijkstra's algorithm) and AON loading with updated costs
        SPTT, x_bar, _, _ = AONloading(graph, zone2centroid, demand, compute_sptt=True)
        # Step 3: Compute the relative gap
        TSTT = sum(graph[u][v]['flow'] * graph[u][v]['cost'] for u, v in graph.edges())

        relative_gap = abs(TSTT - SPTT) / SPTT
        relative_gaps.append(relative_gap)
        
        print(f"Iteration {iteration}: Relative Gap = {relative_gap:.6f} TSTT = {TSTT}")

        # Check for convergence (relative gap < 0.05)
        if relative_gap < convergence_threshold:
            print("Convergence achieved!")
            break

    # Fixed return statement
    return {(u, v): graph[u][v]['flow'] for u, v in graph.edges()}, relative_gaps, graph



def update_edge_costs(graph):
    """
    Update edge travel times (costs) based on current flows using the BPR (Bureau of Public Roads) function.

    Parameters:
        graph (nx.DiGraph): Directed graph representing the network.

    Notes:
        The BPR function is defined as:
            cost = free_flow_time * (1 + alpha * (flow / capacity)^beta)
    """
    alpha = 0.15
    beta = 4
    for u, v, data in graph.edges(data=True):
        free_flow_time = data.get('FFT', 1)
        capacity = data.get('capacity', 1)
        flow = data['flow']
        data['cost'] = free_flow_time * (1 + alpha * (flow / capacity) ** beta)


In [11]:
G_AON = NetGraph.copy()
final_flows, gaps, Assigned_graph = MSA(G_AON, zones2cent, ODdemand)

print("Final Edge Flows:", final_flows)
print("Relative Gaps:", gaps)

Running initial AON assignment...
Iteration 1...
Iteration 1: Relative Gap = 8.819670 TSTT = 67528105.98692872
Iteration 2...
Iteration 2: Relative Gap = 1.475669 TSTT = 21662333.75361345
Iteration 3...
Iteration 3: Relative Gap = 0.960687 TSTT = 19054437.231811553
Iteration 4...
Iteration 4: Relative Gap = 0.345179 TSTT = 13353616.033730421
Iteration 5...
Iteration 5: Relative Gap = 0.202441 TSTT = 11321319.343612956
Iteration 6...
Iteration 6: Relative Gap = 0.148498 TSTT = 10433753.0391537
Iteration 7...
Iteration 7: Relative Gap = 0.120460 TSTT = 9851561.587329142
Iteration 8...
Iteration 8: Relative Gap = 0.105280 TSTT = 9453602.483009612
Iteration 9...
Iteration 9: Relative Gap = 0.097561 TSTT = 9178450.174740136
Iteration 10...
Iteration 10: Relative Gap = 0.083434 TSTT = 9001061.080282092
Iteration 11...
Iteration 11: Relative Gap = 0.071761 TSTT = 8830513.624275452
Iteration 12...
Iteration 12: Relative Gap = 0.068052 TSTT = 8696253.816679358
Iteration 13...
Iteration 13: Rela